In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.1                                                                      #调整学习率为0.1
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 100, entropy los

令人惊喜的是，只经过一次学习率的调整，正确率就已经达到98%了。这里的原因可能是在有限的训练次数下，较高的学习率能够是损失函数更快收敛，使得正确率快速提升。

如果再调高学习率会是什么结果呢？

将学习率调整为0.9

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.9                                                                      
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

step 100, entropy loss: 2.303184, l2_loss: 33526362.000000, total loss: 2349.148682
0.11
step 200, entropy loss: 2.308859, l2_loss: 33106558.000000, total loss: 2319.768066
0.11
step 300, entropy loss: 2.303641, l2_loss: 32692016.000000, total loss: 2290.744873
0.14
step 400, entropy loss: 2.306877, l2_loss: 32282666.000000, total loss: 2262.093506
0.11
step 500, entropy loss: 2.301417, l2_loss: 31878432.000000, total loss: 2233.791748
0.09
step 600, entropy loss: 2.296698, l2_loss: 31479264.000000, total loss: 2205.845215
0.14


KeyboardInterrupt: 

从一开始就出现了很糟糕的结果，应该是很早就出现梯度爆炸的情况，因此再次将学习率调小，调为0.3

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.3                                                                      
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

step 100, entropy loss: 2.294833, l2_loss: 66815.710938, total loss: 6.971933
0.16
step 200, entropy loss: 2.347872, l2_loss: 66535.140625, total loss: 7.005332
0.07
step 300, entropy loss: 2.281934, l2_loss: 66255.953125, total loss: 6.919850
0.13
step 400, entropy loss: 2.316163, l2_loss: 65977.984375, total loss: 6.934623
0.04


KeyboardInterrupt: 

结果仍然不好，因此选择在0.1与0.01之间调参

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.05                                                                      
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.354425, l2_loss: 63709.824219, total loss: 4.814113
0.91
step 200, entropy loss: 0.192688, l2_loss: 63666.246094, total loss: 4.649326
0.93
step 300, entropy loss: 0.321539, l2_loss: 63622.484375, total loss: 4.775113
0.92
step 400, entropy loss: 0.345330, l2_loss: 63578.296875, total loss: 4.795811
0.92
step 500, entropy loss: 0.275506, l2_loss: 63534.351562, total loss: 4.722910
0.89
step 600, entropy loss: 0.127504, l2_loss: 63490.203125, total loss: 4.571819
0.96
step 700, entropy loss: 0.176175, l2_loss: 63446.226562, total loss: 4.617411
0.94
step 800, entropy loss: 0.183268, l2_loss: 63402.101562, total loss: 4.621415
0.97
step 900, entropy loss: 0.149102, l2_loss: 63358.02734

结果相差无几，0.1的学习率要略好一些